In [1]:
import sys
from os.path import abspath,join
sys.path.append(abspath(join('..', 'module')))
from openTable import *

from statistics import mean
from datetime import date
import pandas as pd

In [4]:
def lolos(num, minimum, minimum_atas):
    if num >= minimum_atas:
        return "Kemungkinan Lolos Besar"
    elif num < minimum_atas and num >= minimum:
        return "Kemungkinan Lolos Sedang"
    else:
        return "Kemungkinan Lolos Kecil"

def get_data_jurusan():
    #Get Data Univ
    ptn = pd.read_csv('../data/kampus_x_top.csv')
    ptn = ptn[ptn['Rataan'].notnull()]  
    
    #Get Data Pilihan
    data = open_table_ds(['userId','jurusanId'],'rasionalisasi_pilihan')
    data = pd.DataFrame(data)
    data = data.rename(columns={0:'userId',1:'jurusanId'})

    #Make Dictionary
    result ={}
    for i,j in ptn.iterrows():
        result[int(j['Kode Jurusan'])] = {} 
        result[int(j['Kode Jurusan'])]['jenis'] = j.Jenis
        result[int(j['Kode Jurusan'])]['daya_tampung'] = j['Daya Tampung']
        result[int(j['Kode Jurusan'])]['peminat'] = j.Peminat
        result[int(j['Kode Jurusan'])]['rataan'] = j.Rataan
        result[int(j['Kode Jurusan'])]['s_baku'] = j['S Baku']
        
    groups = data.groupby(['jurusanId','userId'])['jurusanId'].agg({'Frequency':'count'})
    d = {k:list(groups.ix[k].index) for k in groups.index.levels[0]}
    
    #Join Dictionary
    for k in result.keys():
        for l in d.keys():
            if k==l:
                result[k]['userId'] = d[l]
    
    return result

def get_data_pendaftar():  
    data = open_table_ds(['*'],'rasionalisasi_nilai')
    data = pd.DataFrame(data)
    data = data.rename(columns={0:'userId',1:'jenis',2:'penalaran',3:'kuantitatif',
                                4:'umum',5:'baca_tulis',6:'mat_saintek',7:'fisika',
                                8:'kimia',9:'biologi',10:'mat_soshum',11:'geografi',
                                12:'sejarah',13:'sosiologi',14:'ekonomi',15:'rerata'})
    
    return data

def rerata_jurusan(dict_jurusan, data_pendaftar):
    saintek = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_saintek', 'fisika', 'kimia', 'biologi']
    soshum = ['penalaran', 'kuantitatif', 'umum', 'baca_tulis', 'mat_soshum', 'geografi', 'sejarah', 'sosiologi', 'ekonomi']

    for k in dict_jurusan.keys():
        try:
            user = dict_jurusan[k]['userId']
            data = data_pendaftar.loc[data_pendaftar.userId.isin(user)]
            
            if dict_jurusan[k]['jenis'] == 'saintek':
                for i in saintek:
                    dict_jurusan[k][i] = mean(list(data[i]))

            elif dict_jurusan[k]['jenis'] == 'soshum':
                for i in soshum:
                    dict_jurusan[k][i] = mean(list(data[i]))

            dict_jurusan[k]['pendaftar'] = len(data)
            dict_jurusan[k]['last_update'] = date.today()
            
#             print(len(dict_jurusan))
            
        except:
            pass
        
    return dict_jurusan

def logic(dict_jurusan,data_pendaftar):
    
    hasil_list = []
    for k in dict_jurusan:
        try:
            kuota_atas = int(dict_jurusan[k]['daya_tampung']*0.2)
            user = dict_jurusan[k]['userId']
            data = data_pendaftar.loc[data_pendaftar.userId.isin(user)]
            data = data.sort_values('rerata',ascending=False)

            if dict_jurusan[k]['peminat'] > dict_jurusan[k]['daya_tampung']:
#                 n_generate = int(dict_jurusan[k]['peminat'] - dict_jurusan[k]['pendaftar'])
                generate = np.random.normal(dict_jurusan[k]['rataan'], dict_jurusan[k]['s_baku'], dict_jurusan[k]['daya_tampung'])

                gen = pd.DataFrame(generate, columns = ['rerata'])

                sbm = data.append(gen)

                minimum = sbm.nlargest(dict_jurusan[k]['daya_tampung'], 'rerata').tail(1)['rerata'].values[0]
                minimum_atas = sbm.nlargest(kuota_atas, 'rerata').tail(1)['rerata'].values[0]
            else:
                sbm = data

                minimum = 600 #batas bawah
                minimum_atas = 750 #threshold

            sbm = sbm.reset_index(drop=True)
            hasil_n = []
            for index, row in sbm.iterrows():
                hasil_n.append(lolos(row['rerata'], minimum, minimum_atas))

            sbm['hasil'] = hasil_n


            sbm = sbm[np.isfinite(sbm['userId'])]
            sbm['userId'] = sbm['userId'].astype('int')

            for index, row in sbm.iterrows():
                result = {}
                result['userId'] = row['userId']
                result['hasil'] = row['hasil']
                result['ranking'] = index+1
                result['jurusanId'] = k
                hasil_list.append(result)
#             print(len(dict_jurusan))
        except:
            pass
            
    return hasil_list

##def save_db():
    # To Tabel pilihan
    # To Tabel hardcode
    

In [5]:
def batch():
    #Get data univ
    dict_jurusan = get_data_jurusan()
    #Get data pendatfar
    data_pendaftar = get_data_pendaftar()
    #Get nilai rerata jurusan
    dict_jurusan = rerata_jurusan(dict_jurusan, data_pendaftar)
    #Get hasil
    hasil = logic(dict_jurusan,data_pendaftar)
    #Save to DB
    
    return dict_jurusan,data_pendaftar,hasil

In [6]:
dict_jurusan,data_pendaftar,hasil = batch()

In [7]:
hasil

[{'userId': 12347,
  'hasil': 'Kemungkinan Lolos Besar',
  'ranking': 1,
  'jurusanId': 3411017},
 {'userId': 12345,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 2,
  'jurusanId': 3411017},
 {'userId': 123401,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 3,
  'jurusanId': 3411017},
 {'userId': 12346,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 1,
  'jurusanId': 3411346},
 {'userId': 12345,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 2,
  'jurusanId': 3411346},
 {'userId': 123401,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 3,
  'jurusanId': 3411346},
 {'userId': 12347,
  'hasil': 'Kemungkinan Lolos Besar',
  'ranking': 1,
  'jurusanId': 3411354},
 {'userId': 12345,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 2,
  'jurusanId': 3411354},
 {'userId': 123401,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 3,
  'jurusanId': 3411354},
 {'userId': 12345,
  'hasil': 'Kemungkinan Lolos Kecil',
  'ranking': 1,
  'jurusanId': 3411362},
 {'userId': 12345